### Introduction to Machine Learning with Python
## Chapter 7. 텍스트 데이터 다루기
---
## KoNLPY 를 이용한 네이버 영화 리뷰 다루기

### 영화 리뷰 다운로드
- https://github.com/e9t/nsmc
- ratings_train.txt 와 ratings_test.txt 다운로드

### 데이터 읽어오기

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv('data/nsmc/ratings_train.txt', delimiter='\t', 
                       keep_default_na=False)

In [3]:
df_train.shape

(150000, 3)

In [6]:
df_train.head(5) # label 0: 부정, label 1: 긍정

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [7]:
df_test = pd.read_csv('data/nsmc/ratings_test.txt', delimiter='\t', keep_default_na=False)

In [8]:
df_test.shape

(50000, 3)

In [9]:
df_test.head(5)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


### Numpy 어레이로 변환

In [10]:
text_train = df_train.document.values
y_train = df_train.label.values

text_test = df_test.document.values
y_test = df_test.label.values

In [11]:
display(text_train.shape, y_train.shape)
display(text_test.shape, y_test.shape)

(150000,)

(150000,)

(50000,)

(50000,)

In [12]:
np.bincount(y_train)

array([75173, 74827], dtype=int64)

In [13]:
np.bincount(y_test)

array([24827, 25173], dtype=int64)

### konlpy 설치

- http://konlpy.org/ko/latest/install/ 도움말 참조하여 설치 진행
> 32비트와 64비트에 맞게 설치 진행 (아래는 64비트 기준임)

- 자바 jdk 설치

- JAVA_HOME 설정
> C:\Program Files\Java\jdk1.8.0_191

- pip 업그레이드
> (base) c:\khh\서울코딩학원\notebook>python -m pip install --upgrade pip

- JPype1 설치
>- https://www.lfd.uci.edu/~gohlke/pythonlibs/#jpype 에서<br>
> JPype1‑0.6.3‑cp37‑cp37m‑win_amd64.whl 다운로드<br>
>- (base) c:\khh\서울코딩학원\notebook>pip install JPype1-0.6.3-cp37-cp37m-win_amd64.whl

- konlpy 설치
> (base) c:\khh\서울코딩학원\notebook>pip install konlpy

- 아나콘다 프롬프트와 jupyter notebook 재실행

### konlpy 테스트

In [14]:
from konlpy.tag import Twitter, Okt

In [15]:
twitter_tag = Okt() # 기존 Twitter 에서 Okt 로 변경됨

In [18]:
twitter_tag.morphs('겨울이 가고 어느덧 봄이 오는지 어제는 봄비가 많이 내렸습니다')

['겨울', '이', '가고', '어느덧', '봄', '이', '오는지', '어제', '는', '봄비', '가', '많이', '내렸습니다']

### TfidfVectorizer 적용
- 먼저 train, test 각자 1000개씩만 적용

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# help(TfidfVectorizer)

In [20]:
tfidf = TfidfVectorizer(tokenizer=twitter_tag.morphs, min_df=3)
X_train = tfidf.fit_transform(text_train[:1000])
X_test = tfidf.transform(text_test[:1000])

In [21]:
X_train.shape, X_test.shape

((1000, 789), (1000, 789))

In [22]:
display(len(tfidf.get_feature_names()), tfidf.get_feature_names()[::30])

789

['!',
 '??',
 '간',
 '고',
 '근데',
 '남자',
 '다',
 '두',
 '로맨스',
 '멜로',
 '발',
 '볼때',
 '삶',
 '스타',
 '아까운',
 '알았음',
 '에게',
 '영화',
 '으로',
 '이유',
 '자극',
 '전',
 '좋았지만',
 '짓',
 '캐스팅',
 '하는데',
 '현실']

- X_train 에 대해 교차 검증

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

model = LogisticRegression(C=1)
scores = cross_val_score(model, X_train, y_train[:1000], cv=5)
scores

array([0.74626866, 0.74626866, 0.77      , 0.69849246, 0.68844221])

- X_test 예측 점수

In [24]:
model = LogisticRegression(C=1)
model.fit(X_train, y_train[:1000])
score = model.score(X_test, y_test[:1000])
score

0.711

- 모든 데이터 사용

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# tfidf = TfidfVectorizer(tokenizer=twitter_tag.morphs, min_df=3)
# X_train = tfidf.fit_transform(text_train)
# X_test = tfidf.transform(text_test)
# fn = tfidf.get_feature_names()
# np.save('naver_movie.npy', [X_train, X_test])
# np.save('naver_movie_fn.npy', fn)

X_train, X_test = np.load('naver_movie.npy')
fn = np.load('naver_movie_fn.npy')

In [36]:
model = LogisticRegression(C=1)
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
score

0.85182